In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import sys
import pandas as pd
from re import split
from ast import literal_eval
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import numpy as np
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import numpy as np
from re import split
from skimage.filters import gaussian, threshold_otsu
from skimage import measure
from scipy.interpolate import griddata
from scipy.spatial import cKDTree as KDTree
import matplotlib.patches as mpatches
import dask.array as da
from dask import delayed
import dask
from dask_image.ndfilters import uniform_filter as uf
from dask_image.ndmeasure import variance as varian
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.utils import get_pars_from_ini
location = split(', |_|-|!', os.popen('hostname').read())[0].replace("\n", "")
path_data = get_pars_from_ini(campaign='loc')[location]['path_data']
path_proj = get_pars_from_ini(campaign='loc')[location]['path_proj']
plt.style.use('ggplot')

In [2]:
# @jit
def get_col_row(x, size=30):
    ncols = da.ptp(x) / size
    return int(ncols)


def excluding_mesh(x, y, nx=30, ny=30):
    """
    Construct a grid of points, that are some distance away from points (x,
    """

    dx = x.ptp() / nx
    dy = y.ptp() / ny

    xp, yp = np.mgrid[x.min() - 2 * dx:x.max() + 2 * dx:(nx + 2) * 1j,
             y.min() - 2 * dy:y.max() + 2 * dy:(ny + 2) * 1j]
    xp = xp.ravel()
    yp = yp.ravel()

    tree = KDTree(np.c_[x, y])
    dist, j = tree.query(np.c_[xp, yp], k=1)

    # Select points sufficiently far away
    m = (dist > np.hypot(dx, dy))
    return xp[m], yp[m]


def regridd(data, x, y, size=30):
    """
    data = xarray datarray
    size = desired pixel size in meters
    """
    if data.ndim > 2:
        x = da.moveaxis(x.reshape(-1, x.shape[-1]), 0, -1)
        y = da.moveaxis(y.reshape(-1, y.shape[-1]), 0, -1)
        ncols_n = max(np.apply_along_axis(get_col_row, arr=x, axis=1))
        nrows_n = max(np.apply_along_axis(get_col_row, arr=y, axis=1))
        x_new_n = da.from_array(np.moveaxis(np.linspace(np.amin(x, -1), np.amax(x, -1), ncols_n), 0, -1))
        y_new_n = da.from_array(np.moveaxis(np.linspace(np.amax(y, -1), np.amin(y, -1), nrows_n), 0, -1))
        mesh = [delayed(da.meshgrid)(x_new_n[i], y_new_n[i]) for i in range(x_new_n.shape[0])]

        z_n = da.rollaxis(data.reshape(-1, data.shape[-1]), 1)
        idx_n = x.argsort(axis=-1)
        x = np.take_along_axis(x, idx_n, axis=-1)
        y = np.take_along_axis(y, idx_n, axis=-1)
        z_n = np.take_along_axis(z_n, idx_n, axis=-1)

        vp_n = dask.compute(*[delayed(excluding_mesh)(x[i], y[i]) for i in range(x.shape[0])])
        xn = [vp_n[i][0] for i in range(len(vp_n))]
        yn = [vp_n[i][1] for i in range(len(vp_n))]
        zn = dask.compute(*[delayed(da.zeros_like)(xn[i]) for i in range(x.shape[0])])
        xi_ = [mesh[i][0] for i in range(len(vp_n))]
        xi_ = dask.compute(*[da.from_delayed(v, shape=(x.shape[0], np.nan), dtype=float) for v in xi_])
        yi_ = [mesh[i][1] for i in range(len(vp_n))]
        yi_ = dask.compute(*[da.from_delayed(v, shape=(x.shape[0], np.nan), dtype=float) for v in yi_])
        zr = [delayed(griddata)((np.r_[x[i, :], xn[i]], np.r_[y[i, :], yn[i]]), np.r_[z_n[i, :], zn[i]],
                                (xi_[i], yi_[i]), method='linear', fill_value=0)
              for i in range(x.shape[0])]

        zr = da.dstack(dask.compute(*zr))
        xi_ = da.rollaxis(da.rollaxis(da.asarray(xi_), axis=-1), axis=-1)
        yi_ = da.rollaxis(da.rollaxis(da.asarray(yi_), axis=-1), axis=-1)
        return zr, xi_, yi_

    else:
        x_s = x.flatten()
        y_s = y.flatten()
        data = data.compute().flatten()
        idx = x_s.argsort()
        x_s, y_s = np.take_along_axis(x_s, idx, axis=0), np.take_along_axis(y_s, idx, axis=0)
        data = np.take_along_axis(data, idx, axis=0)
        ncols = get_col_row(x=x_s, size=size)
        nrows = get_col_row(x=y_s, size=size)
        x_new = np.linspace(x_s.min(), x_s.max(), int(ncols))
        y_new = np.linspace(y_s.max(), y_s.min(), int(nrows))
        xi, yi = np.meshgrid(x_new, y_new)
        xp, yp = excluding_mesh(x_s, y_s, nx=35, ny=35)
        zp = np.nan + np.zeros_like(xp)
        z0 = griddata((np.r_[x_s, xp], np.r_[y_s, yp]), np.r_[data, zp], (xi, yi), method='linear', fill_value=-9999)
        return z0, xi, yi


def lee_filter_new(img, size, tresh=-150):
    if img.ndim == 2:
        shape = (size, size)
    else:
        shape = (size, size, 1)
    img = da.where(da.logical_or(da.isnan(img), da.equal(img, -9999)), tresh, img)
    img_mean = uf(img, shape)
    img_sqr_mean = uf(da.power(img, 2), shape)
    img_variance = img_sqr_mean - da.power(img_mean, 2)
    overall_variance = varian(img)
    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    img_output = da.where(img_output > 0, img_output, 0)
    return img_output

def process_new(zhh14, _range, azimuth, alt3d, bin_size, time):
    x = _range * bin_size.values[0] * np.sin(np.deg2rad(azimuth))  # add roll
    y = alt3d
    img_filtered = lee_filter_new(zhh14, size=3, tresh=-180)
    img, xi, yi = regridd(img_filtered, x.values, y.values)
    img = np.where(img > 0., img, 0.)
    blurred = gaussian(img, sigma=0.8)
    binary = blurred > threshold_otsu(blurred)
    labels = measure.label(binary)
    if labels.ndim > 2:
        props = [measure.regionprops(labels[:, :, i]) for i in range(labels.shape[-1])]
        _props_all = [[[j.area for j in prop], [j.perimeter for j in prop], [j.major_axis_length for j in prop],
                       [j.minor_axis_length for j in prop], [j.bbox for j in prop]] for prop in props]
        df = pd.DataFrame(data=_props_all, columns=['area', 'perimeter', 'axmax', 'axmin', 'bbox'],
                          index=pd.to_datetime(time))
        max_zhh14 = np.apply_along_axis(np.max, arr=img,  axis=0).compute()
        df['max_zhh14'] = np.apply_along_axis(np.max, arr=max_zhh14,  axis=0)
    else:
        props = measure.regionprops(labels)
        _props_all = [[[prop.area], [prop.perimeter], [prop.major_axis_length], [prop.minor_axis_length],
                       [prop.bbox]] for prop in props]
        df = pd.DataFrame(data=_props_all, columns=['area', 'perimeter', 'axmax', 'axmin', 'bbox'])
        df['max_zhh14'] = np.max(img)

    if img.ndim > 2:
        shp = img.shape[-1]
        for i in range(shp):
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 10))
            img_plot = img[:, :, i]
            ax1.pcolormesh(x.isel(time=i), y.isel(time=i), img_filtered.compute()[:, :, i], cmap='jet',
                           vmax=30, vmin=0, shading='auto')
            ax2.imshow(img_plot, aspect='auto', cmap='jet', vmax=30, vmin=0)
            ax3.pcolormesh(xi[:, :, i], yi[:, :, i], img_plot, cmap='jet',
                           vmax=30, vmin=0, shading='auto')
            for region in props[i]:
                if region.area >= 105:
                    minr, minc, maxr, maxc = region.bbox
                    rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                              fill=False, edgecolor='red', linewidth=2)
                    ax2.add_patch(rect)
            plt.show()

    else:
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 10))
        img_plot = img
        ax1.pcolormesh(x, y, img_filtered.compute(), cmap='jet', vmax=30, vmin=0, shading='auto')
        ax2.imshow(img_plot, aspect='auto', cmap='jet', vmax=30, vmin=0)
        ax3.pcolormesh(xi, yi, img_plot, cmap='jet', vmax=30, vmin=0, shading='auto')
        ax3.set_yticks(np.arange(yi.min(), yi.max(), 250))
        for region in props:
            if region.area >= 100:
                minr, minc, maxr, maxc = region.bbox
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                          fill=False, edgecolor='red', linewidth=2)
                ax2.add_patch(rect)
        plt.show()


In [3]:
cluster = SLURMCluster(queue="seseml",
                       memory='200GB',
                       cores=40,
                       processes=1,
                       walltime='23:40:00',
                       scheduler_options={'host': '172.22.179.3:7227', 'dashboard_address': ':7798'})

In [4]:
# cluster.scale(2)
cluster.adapt(maximum_jobs=4)
cluster

In [5]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://172.22.179.3:7798/status,
Dashboard: http://172.22.179.3:7798/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.22.179.3:7227,Workers: 0
Dashboard: http://172.22.179.3:7798/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [6]:
def correction(df):
    return [float(i) for i in df.replace("[", '').replace("]", '').split()]

In [7]:
df = pd.read_csv('../results/all_filtered_202111190744_zhh90.csv')
df = df[df.max_zhh > 0]

In [8]:
df.rename(columns={'Unnamed: 0': 'time'}, inplace=True)
df['time'] = pd.to_datetime(df.time)
df.index = df.time
df.drop(columns='time', inplace=True)

In [9]:
df_multidx = df[df.columns[9:]]

In [10]:
cols = (literal_eval(i) for i in df_multidx.columns)
df_multidx.columns = pd.MultiIndex.from_tuples(cols)

In [11]:
df_data = df[df.columns[:8]]
df_data = df_data.applymap(correction)
df_data = df_data.merge(df.max_zhh, left_index=True, right_index=True)
df_data.columns = ['area', 'perimeter', 'min_row', 'min_col', 'max_row', 'max_col',
                   'depth', 'width', 'max_zhh']

In [12]:
df_data.head(2)

,area,perimeter,min_row,min_col,max_row,max_col,depth,width,max_zhh
time,,,,,,,,,
2019-09-14 03:26:08.049800,[],[],[],[],[],[],[],[],2.300016
2019-09-14 03:26:20.999800,[],[],[],[],[],[],[],[],4.853605


In [13]:
times = list(df_data.index)

In [14]:
%%time
ds_xr = xr.open_zarr(f'{path_data}/zarr_rckd/KUsKAsWs/lores.zarr')
ds_xr = ds_xr.sel(time=~ds_xr.get_index("time").duplicated())

CPU times: user 247 ms, sys: 12 ms, total: 259 ms
Wall time: 264 ms


In [15]:
ds_data = ds_xr[['alt3D']].sel(time=times)

In [16]:
ds_elevation = ds_data.alt3D.sel(cross_track=12, range=0).to_dataframe()


In [17]:
ds_elevation.head()

,alt3d,cross_track,lat3d,lon3d,alt3D
time,,,,,
2019-09-14 03:26:08.049800,8299.075129,12,15.268955,121.397496,8299.075129
2019-09-14 03:26:20.999800,8320.792632,12,15.261890,121.378783,8320.792632
2019-09-14 03:26:22.849800,8321.293962,12,15.260714,121.376225,8321.293962
2019-09-14 03:26:24.699800,8321.572995,12,15.259453,121.373683,8321.572995
2019-09-14 03:26:26.549800,8322.086630,12,15.258126,121.371082,8322.086630


In [18]:
df_data = df_data.merge(ds_elevation, left_index=True, right_index=True)

In [19]:
df_data.head(2)

,area,perimeter,min_row,min_col,max_row,max_col,depth,width,max_zhh,alt3d,cross_track,lat3d,lon3d,alt3D
time,,,,,,,,,,,,,,
2019-09-14 03:26:08.049800,[],[],[],[],[],[],[],[],2.300016,8299.075129,12,15.268955,121.397496,8299.075129
2019-09-14 03:26:20.999800,[],[],[],[],[],[],[],[],4.853605,8320.792632,12,15.261890,121.378783,8320.792632


In [20]:
df_data.area[0]

[]

In [21]:
df_data = df_data.explode(['area', 'perimeter', 'min_row', 'min_col', 'max_row', 'max_col',
                           'depth', 'width',])

In [22]:
df_data.head(2)

,area,perimeter,min_row,min_col,max_row,max_col,depth,width,max_zhh,alt3d,cross_track,lat3d,lon3d,alt3D
time,,,,,,,,,,,,,,
2019-09-14 03:26:08.049800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.300016,8299.075129,12,15.268955,121.397496,8299.075129
2019-09-14 03:26:20.999800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.853605,8320.792632,12,15.261890,121.378783,8320.792632


In [23]:
df_data = df_data.astype({'area': float, 'perimeter': float, 'depth': float, 'width': float, 'min_row': float, 
                'min_col': float, 'max_row': float, 'max_col': float})

In [24]:
df_data.dtypes

area           float64
perimeter      float64
min_row        float64
min_col        float64
max_row        float64
max_col        float64
depth          float64
width          float64
max_zhh        float64
alt3d          float64
cross_track      int64
lat3d          float64
lon3d          float64
alt3D          float64
dtype: object

In [25]:
df_data = df_data[df_data.area > 0]

In [26]:
df_data.head(5)

,area,perimeter,min_row,min_col,max_row,max_col,depth,width,max_zhh,alt3d,cross_track,lat3d,lon3d,alt3D
time,,,,,,,,,,,,,,
2019-09-14 03:26:22.849800,102.0,46.662951,247.0,64.0,260.0,83.0,18.922731,8.084250,16.600386,8321.293962,12,15.260714,121.376225,8321.293962
2019-09-14 03:26:24.699800,155.0,63.941125,244.0,65.0,260.0,89.0,26.089267,9.647323,23.542559,8321.572995,12,15.259453,121.373683,8321.572995
2019-09-14 03:26:26.549800,181.0,83.183766,243.0,66.0,260.0,101.0,35.712147,9.549721,23.620535,8322.086630,12,15.258126,121.371082,8322.086630
2019-09-14 03:26:26.549800,4.0,4.414214,258.0,153.0,260.0,156.0,3.236068,1.236068,23.620535,8322.086630,12,15.258126,121.371082,8322.086630
2019-09-14 03:26:26.549800,16.0,16.414214,258.0,157.0,260.0,166.0,9.380832,1.984313,23.620535,8322.086630,12,15.258126,121.371082,8322.086630


## initial filters

In [27]:
# df_data = df_data[df_data.area > 105]
df_data = df_data[df_data.width > 0]
df_data = df_data[df_data.depth > 0]

In [28]:
df_data['min_row'] = df_data['min_row'] * 30 / 1000
df_data['max_row'] = df_data['max_row'] * 30 / 1000
df_data['min_col'] = df_data['min_col'] * 30 / 1000
df_data['max_col'] = df_data['max_col'] * 30 / 1000
df_data['area_km2'] = df_data.area * 30 ** 2 / 1000**2
df_data['perimeter_km'] = df_data.perimeter * 30 / 1000 
df_data['depth_km'] = df_data.depth * 30 / 1000 
df_data['width_km'] = df_data.width * 30 / 1000 


In [29]:
df_data.head(2)

,area,perimeter,min_row,min_col,max_row,max_col,depth,width,max_zhh,alt3d,cross_track,lat3d,lon3d,alt3D,area_km2,perimeter_km,depth_km,width_km
time,,,,,,,,,,,,,,,,,,
2019-09-14 03:26:22.849800,102.0,46.662951,7.41,1.92,7.8,2.49,18.922731,8.084250,16.600386,8321.293962,12,15.260714,121.376225,8321.293962,0.0918,1.399889,0.567682,0.242528
2019-09-14 03:26:24.699800,155.0,63.941125,7.32,1.95,7.8,2.67,26.089267,9.647323,23.542559,8321.572995,12,15.259453,121.373683,8321.572995,0.1395,1.918234,0.782678,0.289420


In [30]:
df_data['cloud_top'] =  df_data['alt3D'] / 1000 - df_data['min_row'] 
df_data['cloud_base'] =  df_data['alt3D'] / 1000 - df_data['max_row'] 
df_data['axis_ratio'] = df_data['depth'] / df_data['width']
df_data['axis_ratio_inv'] = df_data['width'] /df_data['depth'] 

In [31]:
df_data['day'] = pd.to_datetime(df_data.index.strftime('%Y-%m-%d'))

In [32]:
df_day = df_data.groupby(df_data.index.floor('d'))
flights = list(df_day.groups.keys())
dict_flights = {f'RF{i + 1:02d}':flights[i] for i in range(len(flights)) }
dict_fl = {flights[i]:f'RF{i + 1:02d}' for i in range(len(flights)) }

In [33]:
df_fl = pd.DataFrame(dict_flights, index=np.arange(0, 25))
df_fl2 = pd.DataFrame(dict_fl, index=np.arange(0, 25))

In [34]:
df_data['vuelo'] = df_data['day'].replace(dict_fl)

## Applying filters

In [35]:
df_data = df_data[df_data.area > 100]
df_data = df_data[df_data.max_zhh < 35]
# # df_data = df_data[df_data.cloud_base > 0.5]
# # condition for finding surface echoes
cond = (df_data.max_zhh > 27.5) & (df_data.axis_ratio_inv < 0.5)
df_data = df_data[~cond]
# # condition on surface echoes disgised as cloudtop
cloud_t = (df_data.max_zhh > 22) & (df_data.area_km2 < 0.5)
df_data = df_data[~cloud_t]

cloud_area = (df_data.area_km2 < 0.1) & (df_data.axis_ratio_inv < 0.4)
df_data = df_data[~cloud_area]

cloud_area = (df_data.area_km2 < 7) & (df_data.max_zhh > 27.5)
df_data = df_data[~cloud_area]

cloud_depth  = (df_data.depth_km > 7) & (df_data.area_km2 < 7)
df_data = df_data[~cloud_depth]

df_data = df_data[df_data.depth_km < 9]
# df_data.shape

# max_ref = ((df_data.max_zhh > 28) & (df_data.axis_ratio_inv < 0.4))
# df_data = df_data[~max_ref]

# max_ref = ((df_data.max_zhh > 43) & (df_data.axis_ratio < 1.5))
# df_data = df_data[~max_ref]


In [36]:
df_data.shape

(4094, 24)

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['area_km2'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ cloud \ area (Km^{2})$')
plt.suptitle('$Cross \ sectional \ cloud \ area$')
plt.title('')
plt.tight_layout()
plt.savefig('../results/area_zhh95.jpg', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['perimeter_km'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
plt.suptitle('$Cross \ sectional \ cloud \ perimeter$')
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ cloud \ perimeter \ (Km)$')
plt.title('')
plt.tight_layout()
plt.savefig('../results/peri_zhh95.jpg', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['depth_km'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ cloud \ depth \ (Km)$')
plt.suptitle('$Cross \ sectional \ cloud \ depth$')
plt.title('')
plt.tight_layout()
plt.savefig('../results/cloud_depth_zhh95.jpg', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['width_km'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ cloud \ width \ (Km)$')
plt.suptitle('$Cross \ sectional \ cloud \ width$')
plt.title('')
plt.tight_layout()

plt.savefig('../results/cloud_width_zhh95.jpg', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['cloud_base'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ cloud \ base \ (Km)$')
plt.suptitle('$Cross \ sectional \ cloud \ base$')
plt.title('')
plt.tight_layout()

plt.savefig('../results/cloud_base_zhh95.jpg')
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['cloud_top'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ cloud \ Top \ (Km)$')
plt.suptitle('$Cross \ sectional \ cloud \ top$')
plt.title('')
plt.tight_layout()

plt.savefig('../results/cloud_top_zhh95.jpg', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['max_zhh'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ maximum \ Ku \ radar \ reflectivity \ (dBZ)$')
plt.suptitle('$Cross \ sectional \ maximum \ radar \ reflectivity$')
plt.title('')
plt.tight_layout()

plt.savefig('../results/radar_ref_zhh95.jpg', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots( figsize=(15, 6))
axis = df_data.boxplot(column=['axis_ratio_inv'], by=['vuelo'], ax=ax)
x_lab = [i for i in dict_flights.keys()]
axis.set_xlabel('$Flight \ Number$')
axis.set_ylabel('$Cross \ sectional \ cloud \ axis \ ratio$')
plt.suptitle('$Axis \ ratio$')
plt.title('')
plt.tight_layout()

plt.savefig('../results/Axis_ratio_inv_zhh95.jpg', dpi=300)
plt.show()

In [ ]:
times = df_data.max_zhh.nlargest(10).index


In [ ]:
for i in times:
    print(i)
    ds_data = ds_xr[['zhh14', 'azimuth', 'DR']].sel(time=i)
    ds_zhh = ds_data.zhh14.where(ds_data.alt3d > 500)
    process_new(zhh14=ds_zhh, _range=ds_data.range, azimuth=ds_data.azimuth,
                alt3d=ds_data.alt3d, bin_size=ds_data.DR, time=ds_data.time)

In [ ]:
df_data.max_zhh.nlargest(20)

In [ ]:
df_data.axis_ratio_inv[df_data.max_zhh.nlargest(20).index]

In [ ]:
df_data.area_km2[df_data.max_zhh.nlargest(20).index]

In [ ]:
df_data.cloud_top[df_data.max_zhh.nlargest(20).index]

In [ ]:
df_data.columns